In [3]:
import copy
import numpy as np
import torch
from tqdm import tqdm
from sklearn.utils import check_random_state
import pandas as pd
from proplace.clfutils import HiddenPrints
import proplace
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# load dataset utils
dataset = proplace.InnDataSet("heloc")

In [5]:
# get a clf#
torch.manual_seed(101)
np.random.seed(1)
random_state = check_random_state(10)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
d = dataset
torch_model = proplace.train_clf(d.X1_train, d.y1_train, d.X1_test, d.y1_test, 20, epochs=50, data_name="heloc", save_clf=False,
                        load_clf=False)
model = proplace.InnModel(dataset, torch_model, 20)

Using cuda device
Epoch 0: train loss: 0.6952842473983765
Epoch 0: train loss: 0.7111822962760925
Epoch 0: train loss: 0.6742647886276245
Epoch 0: train loss: 0.7059459090232849
Epoch 0: train loss: 0.6984533071517944
Epoch 0: train loss: 0.6984993815422058
Epoch 0: train loss: 0.7011096477508545
Epoch 0: train loss: 0.6890342831611633
Epoch 0: train loss: 0.6882796883583069
Epoch 0: train loss: 0.6891630291938782
Epoch 0: train loss: 0.6940578818321228
Epoch 0: train loss: 0.678005576133728
Epoch 0: train loss: 0.6832115650177002
Epoch 0: train loss: 0.6769801378250122
Epoch 0: train loss: 0.6658576130867004
Epoch 0: train loss: 0.6678807139396667
Epoch 0: train loss: 0.6722731590270996
Epoch 0: train loss: 0.7068021893501282
Epoch 0: train loss: 0.6696203947067261
Epoch 0: train loss: 0.6686608195304871
Epoch 0: train loss: 0.6788075566291809
Epoch 0: train loss: 0.6146712899208069
Epoch 0: train loss: 0.6419103741645813
Epoch 0: train loss: 0.6744205355644226
Epoch 0: train loss: 0.

In [6]:
m2s = proplace.retrain_models(dataset, 20, 50) + proplace.retrain_models_leave_some_out(dataset, 20, 50)

100%|██████████| 10/10 [00:59<00:00,  5.92s/it]


## Prepare for exps

In [7]:
from exputils import *
clf = model
test_xs_vals, test_xs, test_xs_carla, utildataset, nodes = get_test_data(model, m2s, dataset)

## Run Baselines

In [13]:
# ours
rnnce_ces, treer, X_class1_clf_robust = run_rnnce(d, clf, nodes, utildataset, 0.025, test_xs_vals)
proplace_ces = run_proplace(clf, nodes, utildataset, 0.025, treer, X_class1_clf_robust, test_xs_vals, k=23)
proplace_scores = eval_ces(clf, m2s, proplace_ces, d, utildataset, test_xs_vals, 0.025)

2437it [01:30, 26.93it/s]
50it [00:22,  2.18it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:07,  6.87it/s]

1.0 0.008631857750211542 >>> robustness - delta validity, avg bound
0.05655135967360235 >>> l1 cost
1.0 >>> percentage of inliers dataset class 1 points
1.0371449953252874 >>> avg lof dataset class 1 points


In [28]:
wach_ces = run_wachter(clf, test_xs_carla)
wach_scores = eval_ces(clf, m2s, wach_ces, d, utildataset, test_xs_vals,  delta=0.025)

[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactual Explanation Found [wachter.py wachter_recourse]
[INFO] Counterfactua

50it [00:06,  7.52it/s]

0.0 -1.2393089529414312 >>> robustness - delta validity, avg bound
0.04541721560210278 >>> l1 cost
0.96 >>> percentage of inliers dataset class 1 points
1.1614613877646853 >>> avg lof dataset class 1 points


In [15]:
rbr_ces = run_rbr(clf, d, random_state, test_xs_vals)
rbr_scores = eval_ces(clf, m2s, rbr_ces, d, utildataset, test_xs_vals, 0.025)

50it [06:34,  7.89s/it]


1.0 >>> validity
0.798 >>> robustness - M2 validity


50it [00:06,  7.37it/s]

0.0 -1.0424438425285794 >>> robustness - delta validity, avg bound
0.037883652215162454 >>> l1 cost
0.96 >>> percentage of inliers dataset class 1 points
1.0979807426319854 >>> avg lof dataset class 1 points


In [16]:
robx_ces = run_robx(clf, d, test_xs_vals)
robx_scores = eval_ces(clf, m2s, robx_ces, d, utildataset, test_xs_vals, 0.025)

2437it [00:07, 342.63it/s]


(1492, 21)


50it [00:02, 22.35it/s]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:06,  7.74it/s]

0.3 -0.15940037399372659 >>> robustness - delta validity, avg bound
0.07321793284645818 >>> l1 cost
1.0 >>> percentage of inliers dataset class 1 points
1.0375535083833867 >>> avg lof dataset class 1 points


In [24]:
roar_params = {
        "lr": 0.008,
        "lambda_": 0.02,
        "delta_max": 0.01,
        "norm": 1,
        "t_max_min": 3,
        "loss_type": "MSE",
        "y_target": [1],
        "binary_cat_features": False,
        "loss_threshold": 1e-3,
        "discretize": False,
        "sample": False,
        "lime_seed": 0,
        "seed": 0,
    }
roar_ces = run_roar(clf, test_xs_carla, roar_params)
roar_scores = eval_ces(clf, m2s, roar_ces, d, utildataset, test_xs_vals, delta=0.025)

[INFO] Start generating LIME coefficients [model.py get_counterfactuals]
[INFO] Finished generating LIME coefficients [model.py get_counterfactuals]
0.98 >>> validity
0.9800000000000002 >>> robustness - M2 validity


50it [00:06,  7.67it/s]

0.9795918367346939 2.490644636973457 >>> robustness - delta validity, avg bound
0.10923561830902402 >>> l1 cost
0.30612244897959184 >>> percentage of inliers dataset class 1 points
1.5730714233034004 >>> avg lof dataset class 1 points


In [27]:
protor_ces = run_proto_r(clf, d, utildataset, test_xs_vals, nodes, delta_target=0.025)
protor_scores = eval_ces(clf, m2s, protor_ces, d, utildataset, test_xs_vals, 0.025)

0it [00:00, ?it/s]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


1it [00:59, 59.66s/it]

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


2it [02:00, 60.55s/it]

...

[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]
[WARNING] No encoder specified. Using k-d trees to represent class prototypes. [cfproto.py fit]


50it [52:01, 62.44s/it]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:06,  7.53it/s]

1.0 0.6957160545189738 >>> robustness - delta validity, avg bound
0.05713099042960892 >>> l1 cost
0.92 >>> percentage of inliers dataset class 1 points
1.2137494690524604 >>> avg lof dataset class 1 points


In [32]:
milpr_ces = run_milpr(clf, d, nodes, utildataset, 0.04, test_xs_vals)
milpr_scores = eval_ces(clf, m2s, milpr_ces, d, utildataset, test_xs_vals, 0.025)

50it [01:59,  2.38s/it]


1.0 >>> validity
1.0 >>> robustness - M2 validity


50it [00:06,  7.67it/s]

1.0 1.27668403822021 >>> robustness - delta validity, avg bound
0.043571706243953505 >>> l1 cost
0.08 >>> percentage of inliers dataset class 1 points
2.483606514151016 >>> avg lof dataset class 1 points


In [33]:
# make table
from tabulate import tabulate
scores_names = ["name", "validity", "delta validity", "M2 validity", "l1", "%inlier class 1", "lof class 1"]
scores_table = [scores_names,
                np.concatenate((['wach'], np.round(wach_scores, 3))),
                np.concatenate((['rbr'], np.round(rbr_scores, 3))),
                np.concatenate((['robx'], np.round(robx_scores, 3))),
                np.concatenate((['roar'], np.round(roar_scores, 3))),
                np.concatenate((['proto-r'], np.round(protor_scores, 3))),
                np.concatenate((['milp-r'], np.round(milpr_scores, 3))),
                np.concatenate((['proplace'], np.round(proplace_scores, 3))), ]
print(tabulate(scores_table, headers='firstrow', tablefmt='outline'))

+----------+------------+------------------+---------------+-------+-------------------+---------------+
| name     |   validity |   delta validity |   M2 validity |    l1 |   %inlier class 1 |   lof class 1 |
+==========+============+==================+===============+=======+===================+===============+
| wach     |       1    |             0    |         0.491 | 0.045 |             0.96  |         1.161 |
| rbr      |       1    |             0    |         0.798 | 0.038 |             0.96  |         1.098 |
| robx     |       1    |             0.3  |         1     | 0.073 |             1     |         1.038 |
| roar     |       0.98 |             0.98 |         0.98  | 0.109 |             0.306 |         1.573 |
| proto-r  |       1    |             1    |         1     | 0.057 |             0.92  |         1.214 |
| milp-r   |       1    |             1    |         1     | 0.044 |             0.08  |         2.484 |
| proplace |       1    |             1    |         1 